<a href="https://www.kaggle.com/code/pranay27sy/model-building-regression-crabage-dataset?scriptVersionId=133009179" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

#models
import xgboost as xgb
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge, TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.model_selection import cross_val_score,RepeatedKFold,GridSearchCV,train_test_split

In [16]:
train = pd.read_pickle('/kaggle/input/feature-engineering-regression-crabage-dataset/train_dataset_001.pkl')

# Rename column with space
train = train.rename(columns={'Shucked Weight': 'Shucked_Weight','Viscera Weight' : 'Viscera_Weight','Shell Weight':'Shell_Weight'})

train.head()

,Length,Diameter,Height,Weight,Shucked_Weight,Viscera_Weight,Shell_Weight,Age,Volume,Density,...,Shucked_by_wt,Viscera_by_wt,Shell_by_wt,Length_into_Height,Length_into_Diameter,Length_into_Weight,L2_weights,Sex_F,Sex_I,Sex_M
id,,,,,,,,,,,,,,,,,,,,,
0,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9,0.406421,71.288527,...,0.439335,0.229452,0.288160,0.571875,1.791875,44.184113,16.611005,0,1,0
1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8,0.146930,70.907553,...,0.434014,0.223129,0.326531,0.302500,0.907500,11.460285,6.117468,0,1,0
2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9,0.364335,68.007353,...,0.457666,0.224256,0.268879,0.520312,1.543594,34.378730,14.277597,0,0,1
3,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11,0.783099,64.692418,...,0.401791,0.216970,0.296027,0.850000,2.401250,86.122946,27.569017,1,0,0
4,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8,0.271602,85.747107,...,0.514303,0.193548,0.255630,0.421875,1.265625,29.111393,14.114728,0,1,0


In [17]:
train.shape

(74051, 22)

In [18]:
X = train.drop(columns=['Age'])
y = train[['Age']].copy()

In [19]:
# # Create an instance of the StandardScaler
# scaler = MinMaxScaler()

# # Fit and transform the data
# X_transformed = scaler.fit_transform(X)
# # Convert the transformed array back to a DataFrame
# X_transformed_df = pd.DataFrame(X_transformed, columns=X.columns)

In [20]:
# #apply SelectKBest class to extract top 10 best features
# bestfeatures = SelectKBest( score_func=chi2, k=10 )
# fit = bestfeatures.fit(X_transformed_df,y)

In [21]:
# dfscores = pd.DataFrame(fit.scores_)
# dfcolumns = pd.DataFrame(X_transformed_df.columns)

In [22]:
# #concat two dataframes for better visualization 
# featureScores = pd.concat([dfcolumns,dfscores],axis=1)
# featureScores.columns = ['Specs','Score']  #naming the dataframe columns

# X = X_transformed_df[featureScores.nlargest(10,'Score')['Specs'].values]
# X.head()

In [23]:
X.shape

(74051, 21)

In [24]:
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

### Model1

In [25]:
# Create a logistic regression model
model_LinearRegression = LinearRegression()
model_ElasticNet = ElasticNet()
model_lasso = Lasso(alpha=0.1)
model_ridge = Ridge(alpha=0.5)
model_ransac = RANSACRegressor()
model_xgb = xgb.XGBRegressor()

In [26]:
# Train the model on the training data
model_LinearRegression.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model_LinearRegression.predict(X_test)

print('MAE : ',mean_absolute_error(y_test, y_pred))
print('r2 : ',r2_score(y_test, y_pred))

MAE :  1.4582950752950707
r2 :  0.5656998484808494


### Xgboost

#### Hyperparameter tuning

In [ ]:
model = xgb.XGBRegressor()

# define grid
grid = dict()
grid['learning_rate'] = np.arange(0.1,1,0.2)
grid['max_depth'] = np.arange(1, 10, 1)
grid['min_child_weight'] = np.arange(1,10,2)
grid['n_estimators'] = [100,200]
grid['subsample'] = [0.7]
grid['gamma'] = [0.1]
grid['reg_lambda'] = [0]
grid['reg_alpha'] = [0]
    
# define search
search = GridSearchCV(model_xgb, grid, scoring='neg_mean_absolute_error', cv=2, n_jobs=-1)
# perform the search
results = search.fit(X_train, y_train)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

In [ ]:
# with open('kaggle/working/best_params.pkl', 'wb') as file:
#     pickle.dump(results.best_params_, file)

In [32]:

model_xgb = xgb.XGBRegressor(objective = 'reg:absoluteerror',learning_rate = 0.1, max_depth = 7,
    min_child_weight = 3, n_estimators = 200, subsample = 0.7, gamma = 0.1, reg_lambda = 0,reg_alpha = 0 )
model_xgb.fit(X_train, y_train)

y_pred = model_xgb.predict(X_test)
mean_absolute_error(y_test, y_pred)

1.3733837448377932

In [ ]:
#model_xgb.get_score(importance_type='gain')

In [34]:
with open('/kaggle/working/model_xgboost.pkl', 'wb') as file:
    pickle.dump(model_xgb, file)
    
with open('/kaggle/working/test_columns.pkl', 'wb') as file:
    pickle.dump(X_train.columns, file)